asjkhsdhkads


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_data(file_path):
    df = pd.read_csv(file_path, sep='\t', names=['pseudocode', 'cpp_code', 'workerid', 'probid', 'subid', 'line', 'indent'])
    df['pseudocode'] = '<sos> ' + df['pseudocode'] + ' <eos>'
    df['cpp_code'] = '<sos> ' + df['cpp_code'] + ' <eos>'
    return df[['pseudocode', 'cpp_code']].dropna()

train_data = load_data('spoc-train-train.tsv')

num_words = 20000
max_len = 150

pseudocode_tokenizer = Tokenizer(num_words=num_words, filters='', lower=True)
pseudocode_tokenizer.fit_on_texts(train_data['pseudocode'])
X_train = pseudocode_tokenizer.texts_to_sequences(train_data['pseudocode'])
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')

cpp_tokenizer = Tokenizer(num_words=num_words, filters='', lower=False)
cpp_tokenizer.fit_on_texts(train_data['cpp_code'])
y_train = cpp_tokenizer.texts_to_sequences(train_data['cpp_code'])
y_train = pad_sequences(y_train, maxlen=max_len, padding='post')




<ipython-input-2-64d4544762f7>:10: DtypeWarning: Columns (2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t', names=['pseudocode', 'cpp_code', 'workerid', 'probid', 'subid', 'line', 'indent'])


In [3]:

with open('pseudocode_tokenizer.pkl', 'wb') as f:
    pickle.dump(pseudocode_tokenizer, f)
with open('cpp_tokenizer.pkl', 'wb') as f:
    pickle.dump(cpp_tokenizer, f)

In [4]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v):
        batch_size = tf.shape(q)[0]
        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)
        attn_output = tf.nn.softmax(tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(self.depth, tf.float32)))
        attn_output = tf.matmul(attn_output, v)
        attn_output = tf.transpose(attn_output, perm=[0, 2, 1, 3])
        return self.dense(tf.reshape(attn_output, (batch_size, -1, self.d_model)))

class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff):
        super(TransformerEncoder, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x):
        attn_output = self.attention(x, x, x)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        return {"d_model": self.d_model, "num_heads": self.num_heads, "dff": self.dff}

class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff):
        super(TransformerDecoder, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.attention1 = MultiHeadAttention(d_model, num_heads)
        self.attention2 = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, enc_output):
        attn1 = self.attention1(x, x, x)
        out1 = self.layernorm1(x + attn1)
        attn2 = self.attention2(out1, enc_output, enc_output)
        out2 = self.layernorm2(out1 + attn2)
        ffn_output = self.ffn(out2)
        return self.layernorm3(out2 + ffn_output)

    def get_config(self):
        return {"d_model": self.d_model, "num_heads": self.num_heads, "dff": self.dff}

@tf.keras.utils.register_keras_serializable()
class Transformer(tf.keras.Model):
    def __init__(self, vocab_size, d_model, num_heads, dff, max_len):
        super(Transformer, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.max_len = max_len

        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, input_length=max_len)
        self.encoder = TransformerEncoder(d_model, num_heads, dff)
        self.decoder = TransformerDecoder(d_model, num_heads, dff)
        self.final_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')

    def call(self, inputs):
        enc_output = self.encoder(self.embedding(inputs))
        dec_output = self.decoder(self.embedding(inputs), enc_output)
        return self.final_layer(dec_output)

    def get_config(self):
        return {
            "vocab_size": self.vocab_size,
            "d_model": self.d_model,
            "num_heads": self.num_heads,
            "dff": self.dff,
            "max_len": self.max_len,
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [7]:
transformer = Transformer(num_words, d_model=128, num_heads=4, dff=512, max_len=max_len)
transformer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

transformer.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Save Transformer Model
transformer.save('transformer_model.keras')

Epoch 1/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 245s 91ms/step - accuracy: 0.9620 - loss: 0.6204 - val_accuracy: 0.9750 - val_loss: 0.1371
Epoch 2/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 257s 90ms/step - accuracy: 0.9777 - loss: 0.1149 - val_accuracy: 0.9760 - val_loss: 0.1275
Epoch 3/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 259s 89ms/step - accuracy: 0.9797 - loss: 0.0959 - val_accuracy: 0.9765 - val_loss: 0.1258
Epoch 4/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 263s 90ms/step - accuracy: 0.9811 - loss: 0.0853 - val_accuracy: 0.9768 - val_loss: 0.1267
Epoch 5/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 263s 90ms/step - accuracy: 0.9820 - loss: 0.0797 - val_accuracy: 0.9769 - val_loss: 0.1273
Epoch 6/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 261s 90ms/step - accuracy: 0.9826 - loss: 0.0764 - val_accuracy: 0.9770 - val_loss: 0.1285
Epoch 7/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 262s 90ms/step - accuracy: 0.9831 - loss: 0.0740 - val_accuracy: 0.9771 - val_loss: 0.1304
Epoch 8/10
2558/2558 ━━━━━━━━━━━━━━━━━━━━ 260s 89ms/step - accuracy: 

In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open('pseudocode_tokenizer.pkl', 'rb') as f:
    pseudocode_tokenizer = pickle.load(f)
with open('cpp_tokenizer.pkl', 'rb') as f:
    cpp_tokenizer = pickle.load(f)

transformer = tf.keras.models.load_model('transformer_model.keras', compile=False)

def generate_cpp_code(pseudocode, max_len=150):
    input_seq = pseudocode_tokenizer.texts_to_sequences(["<sos> " + pseudocode + " <eos>"])
    input_seq = pad_sequences(input_seq, maxlen=max_len, padding='post')

    pred_seq = transformer.predict(input_seq)
    pred_indices = np.argmax(pred_seq, axis=-1)[0]
    cpp_tokens = [cpp_tokenizer.index_word.get(idx, '') for idx in pred_indices if idx > 0]

    return ' '.join(cpp_tokens).replace('<sos>', '').replace('<eos>', '').strip()

pseudocode_example = "read s"
cpp_output = generate_cpp_code(pseudocode_example)
print("Generated C++ Code:\n", cpp_output)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 956ms/step
Generated C++ Code:
 cin >> s;
